### 读取和存储
- 在实际中，我们有时需要把训练好的模型部署到很多不同的设备
- 这种情况下，我们可以把内存中训练好的模型参数存储在硬盘上供后续读取使用

In [10]:
import tensorflow as tf
import numpy as np
print(tf.__version__)
# 可以直接使用save函数和load函数分别存储和读取
x = tf.ones(3)
np.save('../data/x.npy', x)

2.13.0


In [11]:
x2 = np.load('../data/x.npy')
x2

array([1., 1., 1.], dtype=float32)

In [12]:
# 还可以存储一列tensor并读回内存
y = tf.zeros(3)
np.save('../data/xy.npy',[x,y])

In [13]:
x2, y2 = np.load('../data/xy.npy', allow_pickle=True)
(x2, y2)

(array([1., 1., 1.], dtype=float32), array([0., 0., 0.], dtype=float32))

In [14]:
# 甚至可以存储并读取一个从字符串映射到tensor的字典
mydict = {'x': x, 'y': y}
np.save('../data/mydict.npy', mydict)
mydict2 = np.load('../data/mydict.npy', allow_pickle=True)
mydict2

array({'x': <tf.Tensor: shape=(3,), dtype=float32, numpy=array([1., 1., 1.], dtype=float32)>, 'y': <tf.Tensor: shape=(3,), dtype=float32, numpy=array([0., 0., 0.], dtype=float32)>},
      dtype=object)

In [15]:
!ls ../data

1-s2.0-S0167732222024916-mmc1.docx    inhibitor_data.csv
amino_acids_hydrate_data.csv	      kaggle_house
amino_acids_hydrate_data_update1.csv  mydict.npy
bioactivity_preprocessed_data.csv     x.npy
house_tiny.csv			      xy.npy
inhibitor_amino_acids_data.csv


In [16]:
# 还可以读写模型的参数
X = tf.random.normal((2,20))
class MLP(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.flatten = tf.keras.layers.Flatten()    # Flatten层将除第一维（batch_size）以外的维度展平
        self.dense1 = tf.keras.layers.Dense(units=256, activation=tf.nn.relu)
        self.dense2 = tf.keras.layers.Dense(units=10)

    def call(self, inputs):         
        x = self.flatten(inputs)   
        x = self.dense1(x)    
        output = self.dense2(x)     
        return output

net = MLP()
Y = net(X)
Y


<tf.Tensor: shape=(2, 10), dtype=float32, numpy=
array([[-0.10626581, -0.07222503, -0.11006147,  0.19855481,  0.34686363,
        -0.02390967, -0.64972293,  0.1463188 , -0.3179079 , -0.14297329],
       [ 0.33284116,  0.3321502 , -0.02858605, -0.4042737 ,  0.03034445,
         0.27789673, -0.5233873 ,  0.37224627, -0.01023898,  0.26687536]],
      dtype=float32)>

In [17]:
# 把该模型的参数存成文件
net.save_weights("../data/4.5saved_model.h5")


In [18]:
# 再实例化一次定义好的多层感知机
# 直接读取保存在文件里的参数
net2 = MLP()
net2(X)
net2.load_weights("../data/4.5saved_model.h5")
Y2 = net2(X)
Y2 == Y


<tf.Tensor: shape=(2, 10), dtype=bool, numpy=
array([[ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True]])>